In [40]:
# Import all Data Science libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [41]:
sms_spam = pd.read_csv('SMSSpamCollection', sep='\t', names=['Label', 'SMS'])
# sep='\t' is used to specify that the columns are separated by tabs
# names=['Label', 'SMS'] is used to specify the column names

print(sms_spam.shape)
sms_spam.head()

(5572, 2)


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


# Proporção entre Ham e Spam

In [42]:
distribuicao = sms_spam['Label'].value_counts(normalize=True)
print("Distribuição original:")
print(distribuicao)

Distribuição original:
Label
ham     0.865937
spam    0.134063
Name: proportion, dtype: float64


Como podemos ver a distribuição esta com 86% de ham e 14% de spam </br>
Mas de acordo com algumas pesquisas, vi que atualmente, aproximadamente
50% dos emails que recebemos é spam.</br>
Por isso faremos uma manipulação para transformar em uma amostra 50-50.

In [43]:
# Divida o DataFrame em dois com base no tipo
spam_df = sms_spam[sms_spam['Label'] == 'spam']
ham_df = sms_spam[sms_spam['Label'] == 'ham']

# Amostra aleatória do DataFrame ham para ter o mesmo número de linhas que o DataFrame spam
ham_amostrado = ham_df.sample(n=len(spam_df), replace=False)

# Concatenar os DataFrames amostrados
df_balanceado = pd.concat([spam_df, ham_amostrado])

# Verifique a distribuição após a amostragem
nova_distribuicao = df_balanceado['Label'].value_counts(normalize=True)
print("\nNova distribuição:")
print(nova_distribuicao)




Nova distribuição:
Label
spam    0.5
ham     0.5
Name: proportion, dtype: float64


In [44]:
df_balanceado

,Label,SMS
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
5,spam,FreeMsg Hey there darling it's been 3 week's n...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...
11,spam,"SIX chances to win CASH! From 100 to 20,000 po..."
...,...,...
5290,ham,Dear where you. Call me
4559,ham,PISS IS TALKING IS SOMEONE THAT REALISE U THAT...
3900,ham,Ü mean it's confirmed... I tot they juz say on...
3949,ham,I like to think there's always the possibility...


# Treino e Teste

Agora faremos a divisão do dataframe entre treino e teste</br>
Usaremos 80% para treino e 20% para teste</br>

Faremos uma randomização e garantir que as mensagens de ham e spam estejam bem distribuidas pelo dataframe 

In [45]:
# randomizar o dataset
df_randomized = df_balanceado.sample(frac=1, random_state=1)

# separar entre treino e teste
# calcula o índice de corte
train_index = round(len(df_randomized) * 0.8)
# divide o dataset
train = df_randomized[:train_index].reset_index(drop=True)
test = df_randomized[train_index:].reset_index(drop=True)

In [46]:
train.shape, test.shape

((1195, 2), (299, 2))

Agora vamos analisar a porcentagem de ham e spam pra ver se a proporção continua a mesma

In [47]:
train['Label'].value_counts(normalize=True), test['Label'].value_counts(normalize=True)

(Label
 spam    0.503766
 ham     0.496234
 Name: proportion, dtype: float64,
 Label
 ham     0.51505
 spam    0.48495
 Name: proportion, dtype: float64)

# Tratamento dos Dados

Vamos utilizar o algoritimo de Naive Bayes, que funciona calculando a probabilidade da mensagem ser ham ou spam a partir das palavras utilizadas.E para fazer esse calculo precisamos fazer um tratamento nos dados.

In [48]:
train.head(5)

,Label,SMS
0,spam,Todays Vodafone numbers ending with 4882 are s...
1,spam,"URGENT!: Your Mobile No. was awarded a £2,000 ..."
2,ham,Ok ill tell the company
3,spam,Fantasy Football is back on your TV. Go to Sky...
4,ham,But you were together so you should be thinkin...


Primeiro vamos remover todas as pontuações e entao colocar todas as letras minusculas


In [49]:
# Remover todos os caracteres que não são letras, números ou espaços
train['SMS'] = train['SMS'].str.replace('\W', ' ')
# Colocar todas as palavras em minúsculas
train['SMS'] = train['SMS'].str.lower()

In [50]:
train.head(5)

,Label,SMS
0,spam,todays vodafone numbers ending with 4882 are s...
1,spam,"urgent!: your mobile no. was awarded a £2,000 ..."
2,ham,ok ill tell the company
3,spam,fantasy football is back on your tv. go to sky...
4,ham,but you were together so you should be thinkin...


Em seguida trocar a coluna SMS por diversas colunas contendo todas as palavras unicas contidas no texto e entao contar quantas vezes essas palavras aparecem nos textos.

In [51]:
# criar um vocabulário com todas as palavras
vocabulary = []
for sms in train['SMS']:
    for word in sms.split():
        vocabulary.append(word)

vocabulary = list(set(vocabulary))

In [52]:
len(vocabulary)

5577

In [53]:
train.head(5)

,Label,SMS
0,spam,todays vodafone numbers ending with 4882 are s...
1,spam,"urgent!: your mobile no. was awarded a £2,000 ..."
2,ham,ok ill tell the company
3,spam,fantasy football is back on your tv. go to sky...
4,ham,but you were together so you should be thinkin...


In [56]:
# criamos um dicionário com todas as palavras do vocabulário e um contador zerado para cada SMS
word_counts_per_sms = {unique_word: [0] * len(train['SMS']) for unique_word in vocabulary}
# preenchemos o dicionário com a contagem de cada palavra
for index, sms in enumerate(train['SMS']):
    for word in sms.split():
        word_counts_per_sms[word][index] += 1


In [57]:
# criar um dataframe com o dicionário
word_counts = pd.DataFrame(word_counts_per_sms)

In [58]:
word_counts.head(5)

,delivery,09058091854,popped,legal.,max10mins,symbol,vl,txt,shows,sonyericsson,...,explicit,oranges,console.,sec?,subs,taunton,hg/suite342/2lands,had..ya,who,12:30
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
# concatenar os dataframes e junta label, sms e contagem de palavras
train_clean = pd.concat([train, word_counts], axis=1)
train_clean.head(5)

,Label,SMS,delivery,09058091854,popped,legal.,max10mins,symbol,vl,txt,...,explicit,oranges,console.,sec?,subs,taunton,hg/suite342/2lands,had..ya,who,12:30
0,spam,todays vodafone numbers ending with 4882 are s...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,spam,"urgent!: your mobile no. was awarded a £2,000 ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,ok ill tell the company,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,spam,fantasy football is back on your tv. go to sky...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,but you were together so you should be thinkin...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Naive Bayes

Teorema de Bayes no contexto dos filtros de spam usando o método chamado de **suavização de Laplace**</br>
Esta fórmula é usada para calcular a probabilidade de uma palavra $w_i$ aparecer em um email dado que este email é spam, denotado como $P(w_i|\text{Spam})$.

Vamos decompor a fórmula para entender cada parte:
$$P(w_i|\text{Spam}) = \frac{N_{w_i|\text{Spam}} + \alpha}{N_{\text{Spam}} + \alpha \cdot N_{\text{vocabulary}}}$$

- $P(w_i|\text{Spam})$: é a probabilidade de ocorrer a palavra $w_i$ em um email sabendo que esse email é spam.
- $N_{w_i|\text{Spam}}$: é o número de vezes que a palavra $w_i$ aparece em emails marcados como spam.
- $N_{\text{Spam}}$: é o número total de palavras em todos os emails marcados como spam.
- $N_{\text{vocabulary}}$: é o número total de palavras únicas em todos os emails, tanto spam quanto ham. Isso é conhecido como o tamanho do vocabulário.
- $\alpha$: é um parâmetro de suavização, usado para lidar com palavras que não aparecem nos emails de spam durante o treinamento. Normalmente, $\alpha$ é um número pequeno e positivo, como 1. Isso é conhecido como suavização de Laplace.

A suavização de Laplace é importante porque evita que \(P(w_i|\text{Spam})\) seja zero para palavras que não foram vistas no treinamento mas aparecem em um novo email.


In [62]:
# Isolando as mensagens de spam e ham
spam_messages = train_clean[train_clean['Label'] == 'spam']
ham_messages = train_clean[train_clean['Label'] == 'ham']

#Probabilidade de spam e ham
p_spam = len(spam_messages) / len(train_clean)
p_ham = len(ham_messages) / len(train_clean)

# Número de palavras em todas as mensagens de spam
n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_span = n_words_per_spam_message.sum()

# Número de palavras em todas as mensagens de ham
n_words_per_ham_message = ham_messages['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()

# Número de palavras no vocabulário
n_vocabulary = len(vocabulary)

# Laplace smoothing
alpha = 1

In [65]:
# iniciar dicionários para spam e ham
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# calcular as probabilidades
for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum()
    p_word_given_spam = (n_word_given_spam + alpha) / (n_span + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    
    n_word_given_ham = ham_messages[word].sum()
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham

Probabilidade de cada palavra pertencer a um email spam

In [66]:
parameters_spam

{'delivery': 0.0001682274435036169,
 '09058091854': 3.3645488700723376e-05,
 'popped': 1.1215162900241126e-05,
 'legal.': 1.1215162900241126e-05,
 'max10mins': 5.607581450120563e-05,
 'symbol': 2.2430325800482252e-05,
 'vl': 1.1215162900241126e-05,
 'txt': 0.0012785285706274883,
 'shows': 0.0003140245612067515,
 'sonyericsson': 4.4860651600964504e-05,
 'manege': 1.1215162900241126e-05,
 'unsubscribed': 2.2430325800482252e-05,
 'mustprovide': 2.2430325800482252e-05,
 'nigeria': 1.1215162900241126e-05,
 'valid12hrs': 2.2430325800482252e-05,
 '08719181259': 2.2430325800482252e-05,
 'purchases': 1.1215162900241126e-05,
 '4goten': 2.2430325800482252e-05,
 'mind': 1.1215162900241126e-05,
 'close': 2.2430325800482252e-05,
 'jamster.co.uk!': 2.2430325800482252e-05,
 'regular': 2.2430325800482252e-05,
 '08712400200.': 2.2430325800482252e-05,
 'saristar': 2.2430325800482252e-05,
 'him': 1.1215162900241126e-05,
 '"sleep': 1.1215162900241126e-05,
 'e.': 3.3645488700723376e-05,
 'friendship': 1.121

Probabilidade de cada palavra pertencer a um email ham

In [67]:
parameters_ham

{'delivery': 2.093714668564968e-05,
 '09058091854': 2.093714668564968e-05,
 'popped': 4.187429337129936e-05,
 'legal.': 4.187429337129936e-05,
 'max10mins': 2.093714668564968e-05,
 'symbol': 4.187429337129936e-05,
 'vl': 8.374858674259872e-05,
 'txt': 2.093714668564968e-05,
 'shows': 4.187429337129936e-05,
 'sonyericsson': 2.093714668564968e-05,
 'manege': 4.187429337129936e-05,
 'unsubscribed': 2.093714668564968e-05,
 'mustprovide': 2.093714668564968e-05,
 'nigeria': 8.374858674259872e-05,
 'valid12hrs': 2.093714668564968e-05,
 '08719181259': 2.093714668564968e-05,
 'purchases': 4.187429337129936e-05,
 '4goten': 2.093714668564968e-05,
 'mind': 6.281144005694904e-05,
 'close': 4.187429337129936e-05,
 'jamster.co.uk!': 2.093714668564968e-05,
 'regular': 2.093714668564968e-05,
 '08712400200.': 2.093714668564968e-05,
 'saristar': 2.093714668564968e-05,
 'him': 0.00037686864034169426,
 '"sleep': 4.187429337129936e-05,
 'e.': 2.093714668564968e-05,
 'friendship': 4.187429337129936e-05,
 'wo

## Criar um classificador de mensagem

In [68]:
import re

# criar uma função para classificar as mensagens
def classificador(message):
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_ham_given_message = p_ham
    p_spam_given_message = p_spam
    
    #Calcula a probabilidade de ham e spam
    for word in message:
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
            
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
    # mostra a probabilidade de ham e spam
    print('P(Ham|message):', p_ham_given_message)
    print('P(Spam|message):', p_spam_given_message)
    
    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Não consigo, classifique voce mesmo')
    
    

In [79]:
classificador("You earned 100,000 points in your loyalty program")

P(Ham|message): 1.347365690671969e-22
P(Spam|message): 1.8619981043678517e-22
Label: Spam


# Testando a Acuracia do Filtro

Primeiro fazemos a função para classificar o dados de teste e adicionar uma coluna nova com a predição feita

In [73]:
# função para classificar todas as mensagens
def classificador_test_set(message):
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_ham_given_message = p_ham
    p_spam_given_message = p_spam
    
    for word in message:
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
            
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
    
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'
    
    
# aplicar a função para classificar todas as mensagens e criar uma coluna com o resultado
test['predicted'] = test['SMS'].apply(classificador_test_set)
test.head(5)

,Label,SMS,predicted
0,ham,Hey. What happened? U switch off ur cell d who...,ham
1,ham,I liked your new house,spam
2,ham,Should I have picked up a receipt or something...,ham
3,ham,He also knows about lunch menu only da. . I know,ham
4,spam,You've won tkts to the EURO2004 CUP FINAL or £...,spam


## Calcular quais mensagens foram preditas corretamente

In [74]:
# calcular a precisão
correct = 0
total = test.shape[0]

for row in test.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1

print('Correto:', correct)
print('Incorreto:', total - correct)
print('Acuracia:', correct/total)

Correto: 286
Incorreto: 13
Acuracia: 0.9565217391304348
